**Table of contents**<a id='toc0_'></a>    
- [Linear regression](#toc1_1_1_)    
    - [Decision Tree regressor](#toc1_1_2_)    
    - [Random Forest regressor](#toc1_1_3_)    
    - [XGBoost regressor](#toc1_1_4_)    
    - [Light GBM regressor](#toc1_1_5_)    
    - [Observations:](#toc1_1_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
# conda install -c conda-forge lightgbm

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
import pickle

In [2]:
train = pd.read_csv("train_FE.csv")
train

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_hour,...,pickup_year,pickup_day_of_week,hav_distance,euc_distance,precipitation,snow_depth,snowfall,max_temp,min_temp,avg_wind
0,0,26:21.0,4.5,6/15/2009 17:26,-73.844311,40.721319,-73.841610,40.712278,1,17,...,2009,Monday,0.640487,0.651073,127,0,0,222,139,14
1,1,52:16.0,16.9,1/5/2010 16:52,-74.016048,40.711303,-73.979268,40.782004,1,16,...,2010,Tuesday,5.250670,5.499001,0,0,0,-11,-67,44
2,2,35:00.0,5.7,8/18/2011 0:35,-73.982738,40.761270,-73.991242,40.750562,2,0,...,2011,Thursday,0.863411,0.943509,18,0,0,283,194,22
3,3,30:42.0,7.7,4/21/2012 4:30,-73.987130,40.733143,-73.991567,40.758092,1,4,...,2012,Saturday,1.739386,1.748493,56,0,0,217,122,22
4,4,51:00.0,5.3,3/9/2010 7:51,-73.968095,40.768008,-73.956655,40.783762,1,7,...,2010,Tuesday,1.242218,1.343397,0,0,0,161,67,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1048570,08:42.0,4.9,5/12/2009 18:08,-74.002872,40.739318,-74.003796,40.748139,1,18,...,2009,Tuesday,0.611389,0.611979,0,0,0,206,100,15
1048571,1048571,25:03.0,14.1,11/1/2010 4:25,-73.978607,40.783016,-73.990416,40.740501,1,4,...,2010,Monday,3.001808,3.044595,0,0,0,94,28,26
1048572,1048572,35:23.0,15.5,6/21/2014 1:35,-73.997814,40.763970,-73.992577,40.727597,4,1,...,2014,Saturday,2.528039,2.535617,0,0,0,256,167,22
1048573,1048573,11:00.0,10.0,11/1/2014 15:11,-73.972843,40.749210,-73.986353,40.750092,1,15,...,2014,Saturday,0.709774,0.934174,89,0,0,83,56,40


In [3]:
train=train.drop('Unnamed: 0', axis=1)

In [4]:
train

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_hour,pickup_day,...,pickup_year,pickup_day_of_week,hav_distance,euc_distance,precipitation,snow_depth,snowfall,max_temp,min_temp,avg_wind
0,26:21.0,4.5,6/15/2009 17:26,-73.844311,40.721319,-73.841610,40.712278,1,17,15,...,2009,Monday,0.640487,0.651073,127,0,0,222,139,14
1,52:16.0,16.9,1/5/2010 16:52,-74.016048,40.711303,-73.979268,40.782004,1,16,5,...,2010,Tuesday,5.250670,5.499001,0,0,0,-11,-67,44
2,35:00.0,5.7,8/18/2011 0:35,-73.982738,40.761270,-73.991242,40.750562,2,0,18,...,2011,Thursday,0.863411,0.943509,18,0,0,283,194,22
3,30:42.0,7.7,4/21/2012 4:30,-73.987130,40.733143,-73.991567,40.758092,1,4,21,...,2012,Saturday,1.739386,1.748493,56,0,0,217,122,22
4,51:00.0,5.3,3/9/2010 7:51,-73.968095,40.768008,-73.956655,40.783762,1,7,9,...,2010,Tuesday,1.242218,1.343397,0,0,0,161,67,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,08:42.0,4.9,5/12/2009 18:08,-74.002872,40.739318,-74.003796,40.748139,1,18,12,...,2009,Tuesday,0.611389,0.611979,0,0,0,206,100,15
1048571,25:03.0,14.1,11/1/2010 4:25,-73.978607,40.783016,-73.990416,40.740501,1,4,1,...,2010,Monday,3.001808,3.044595,0,0,0,94,28,26
1048572,35:23.0,15.5,6/21/2014 1:35,-73.997814,40.763970,-73.992577,40.727597,4,1,21,...,2014,Saturday,2.528039,2.535617,0,0,0,256,167,22
1048573,11:00.0,10.0,11/1/2014 15:11,-73.972843,40.749210,-73.986353,40.750092,1,15,1,...,2014,Saturday,0.709774,0.934174,89,0,0,83,56,40


In [5]:
# Considering all the Features except 'Key', 'pickup_date', 'pickup_day_of_week' and 'euc_distance' (as haversian distance is already being considered, and both together could lead to redundant features.)
X = train[train.columns[3:]]
X = X[X.columns.difference(['pickup_day_of_week'])]
X = X[X.columns.difference(['pickup_date'])]
X = X[X.columns.difference(['euc_distance'])]

# Trainoutput is the Fare Amount column
y = train[train.columns[1:2]]

In [6]:
X

,avg_wind,dropoff_latitude,dropoff_longitude,hav_distance,max_temp,min_temp,passenger_count,pickup_day,pickup_hour,pickup_latitude,pickup_longitude,pickup_month,pickup_year,precipitation,snow_depth,snowfall
0,14,40.712278,-73.841610,0.640487,222,139,1,15,17,40.721319,-73.844311,6,2009,127,0,0
1,44,40.782004,-73.979268,5.250670,-11,-67,1,5,16,40.711303,-74.016048,1,2010,0,0,0
2,22,40.750562,-73.991242,0.863411,283,194,2,18,0,40.761270,-73.982738,8,2011,18,0,0
3,22,40.758092,-73.991567,1.739386,217,122,1,21,4,40.733143,-73.987130,4,2012,56,0,0
4,20,40.783762,-73.956655,1.242218,161,67,1,9,7,40.768008,-73.968095,3,2010,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,15,40.748139,-74.003796,0.611389,206,100,1,12,18,40.739318,-74.002872,5,2009,0,0,0
1048571,26,40.740501,-73.990416,3.001808,94,28,1,1,4,40.783016,-73.978607,11,2010,0,0,0
1048572,22,40.727597,-73.992577,2.528039,256,167,4,21,1,40.763970,-73.997814,6,2014,0,0,0
1048573,40,40.750092,-73.986353,0.709774,83,56,1,1,15,40.749210,-73.972843,11,2014,89,0,0


In [7]:
y

,fare_amount
0,4.5
1,16.9
2,5.7
3,7.7
4,5.3
...,...
1048570,4.9
1048571,14.1
1048572,15.5
1048573,10.0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(734002, 16) (734002, 1)
(314573, 16) (314573, 1)


* We can use 3 Error Metrics to evaluate our models.<br>
    ○ Root Mean Square Error (RMSE)<br>
    ○ Mean Squared Error (MSE)<br>
    ○ R2 Score

* Our goal is to try to maximize R2 score (1 being the best) while minimizing RMSE and MSE along with ensuring that the model does not overfit.

### <a id='toc1_1_1_'></a>[Linear regression](#toc0_)

In [9]:
lr= LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [10]:
# perform k-fold cross-validation
scores = cross_val_score(lr, X_train, y_train, cv=6)

# predict on test set and calculate metrics
lr_y_pred = cross_val_predict(lr, X_test, y_test, cv=6)

In [11]:
# The coefficients
print('Coefficients: \n', lr.coef_)

# ERROR METRICS

# Root Mean Square Error
rmse = np.sqrt(mean_squared_error(y_test, lr_y_pred))
print("RMSE: %f" % (rmse))

# The Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(y_test, lr_y_pred))

# R2 Score
print('Variance score: %.2f' % r2_score(y_test, lr_y_pred))

Coefficients: 
 [[ 5.81858054e-04 -1.76509060e+01 -2.51490728e+00  3.56815646e+00
  -1.04831897e-03  3.53959944e-04  3.26941267e-02  1.22030883e-03
   8.82547694e-03  4.68509188e+00  1.80382081e+01  7.51013839e-02
   5.04119665e-01  3.73168060e-04  8.88572900e-06 -9.38693735e-04]]
RMSE: 4.591985
Mean squared error: 21.09
Variance score: 0.76


* Linear regression model achieved an RMSE = 4.51, MSE = 20.39 and R2 score = 0.77.

* For a basic linear model (that could be considered as the baseline model) I believe that it did quite okay providing an RMSE = 4.62 on the Test Data. But, this is definitely not a good model and has to be improved upon.

* Also, I think for data as sparse and vast as that we have, we may not be able to model it linearly as it involves a lot of real world parameters and in general might contain lots of outliers along with many other underlying dependencies among the features. Hence, linear regression did not produce that good a result for the problem.

### <a id='toc1_1_2_'></a>[Decision Tree regressor](#toc0_)

In [12]:
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

DecisionTreeRegressor()

In [13]:
# perform k-fold cross-validation
scores = cross_val_score(dt, X_train, y_train, cv=6)

# predict on test set and calculate metrics
dt_y_pred = cross_val_predict(dt, X_test, y_test, cv=6)

In [14]:
# ERROR METRICS

# Root Mean Square Error
rmse_dt = np.sqrt(mean_squared_error(y_test, dt_y_pred))
print("RMSE: %f" % (rmse_dt))

# The Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(y_test, dt_y_pred))

# R2 Score
print('Variance score: %.2f' % r2_score(y_test, dt_y_pred))

RMSE: 5.447574
Mean squared error: 29.68
Variance score: 0.67


### <a id='toc1_1_3_'></a>[Random Forest regressor](#toc0_)

In [15]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

/Users/rohithreddykota/miniconda3/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [16]:
rf_y_pred = rf.predict(X_test)

In [17]:
# ERROR METRICS

# Root Mean Square Error
rmse_rf = np.sqrt(mean_squared_error(y_test, rf_y_pred))
print("RMSE: %f" % (rmse_rf))

# The Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(y_test, rf_y_pred))

# R2 Score
print('Variance score: %.2f' % r2_score(y_test, rf_y_pred))

RMSE: 3.739248
Mean squared error: 13.98
Variance score: 0.84


### <a id='toc1_1_4_'></a>[XGBoost regressor](#toc0_)

In [18]:
xgbr = xgb.XGBRegressor()
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [19]:
# perform k-fold cross-validation
scores = cross_val_score(xgbr, X_train, y_train, cv=6)

# predict on test set and calculate metrics
xgbr_y_pred = cross_val_predict(xgbr, X_test, y_test, cv=6)

In [20]:
# ERROR METRICS

# Root Mean Square Error
rmse_xgbr = np.sqrt(mean_squared_error(y_test, xgbr_y_pred))
print("RMSE: %f" % (rmse_xgbr))

# The Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(y_test, xgbr_y_pred))

# R2 Score
print('Variance score: %.2f' % r2_score(y_test, xgbr_y_pred))

RMSE: 3.734814
Mean squared error: 13.95
Variance score: 0.84


In [21]:
#Hyperparameter tuning

#set parameters for xgboost
params = {'max_depth':[7,8,9,10],
          'eta':[1],
          'silent':[1],
          'objective':['reg:linear'],
          'eval_metric':['rmse'],
          'learning_rate':[0.1,0.15,0.2]
         }

# Create the grid search object
grid_search = GridSearchCV(estimator=xgbr, param_grid=params, cv=5, n_jobs=-1)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

xgbr2_y_pred = grid_search.predict(X_test)

/Users/rohithreddykota/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:18:14] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/Users/rohithreddykota/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:18:14] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/rohithreddykota/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:18:14] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/Users/rohithreddykota/miniconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:18:14] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: {

In [22]:
# ERROR METRICS

# Root Mean Square Error
rmse_xgbr2 = np.sqrt(mean_squared_error(y_test, xgbr2_y_pred))
print("RMSE: %f" % (rmse_xgbr2))

# The Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(y_test, xgbr2_y_pred))

# R2 Score
print('Variance score: %.2f' % r2_score(y_test, xgbr2_y_pred))

RMSE: 3.648312
Mean squared error: 13.31
Variance score: 0.85


### <a id='toc1_1_5_'></a>[Light GBM regressor](#toc0_)

In [23]:
lgbm = LGBMRegressor (boosting_type='gbdt', num_leaves=31, learning_rate=0.1, max_depth=5, n_estimators=100, silent = True)
lgbm.fit (X_train, y_train)

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016769 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1859
[LightGBM] [Info] Number of data points in the train set: 734002, number of used features: 16
[LightGBM] [Info] Start training from score 11.284355
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

LGBMRegressor(max_depth=5, silent=True)

In [24]:
lgbm_y_predict = lgbm.predict(X_test)

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [25]:
# ERROR METRICS

# Root Mean Square Error
rmse_lbgm = np.sqrt(mean_squared_error(y_test, lgbm_y_predict))
print("RMSE: %f" % (rmse_lbgm))

# The Mean Squared Error
print("Mean squared error: %.2f" % mean_squared_error(y_test, lgbm_y_predict))

# R2 Score
print('Variance score: %.2f' % r2_score(y_test, lgbm_y_predict))

RMSE: 3.760545
Mean squared error: 14.14
Variance score: 0.84


### <a id='toc1_1_6_'></a>[Observations:](#toc0_)

● The RMSE score for the different models on the Test Data was as follows:<br>
    ○ Linear Regression - RMSE score = 4.671866<br>
    ○ Decision Trees - RMSE score = 5.423396 (was worse than Linear Model)<br>
    ○ Random Forest Regressor - RMSE score = 3.733577 (drastic improvement from here on)<br>
    ○ XGB Regressor with Parameters - RMSE score after tuning the parameters few times = 3.611114<br>
    ○ LGBM Regressor - RMSE score = 3.810423 (worse than XBGR)<br>

● We can clearly see that XGBoost regressor turned out to be the best machine learning model which is capable of determining the taxi fare with most accuracy.

● Real world datasets are extremely complicated and visualizing this kind of data in order to understand the data better requires efforts. In other words, a simple linear model is not always sufficient to best model a data BUT should definitely be the way to start as it gives us a better understanding into the data spread and feature dependencies.

In [26]:
result_df = pd.DataFrame(data={'Model': ['Linear regression', 'Decision Tree', 'Random forest', 'XGBoost', 'Light GBM'], 'RMSE': [rmse, rmse_dt, rmse_rf, rmse_xgbr2, rmse_lbgm], 'MSE': [mean_squared_error(y_test, lr_y_pred), mean_squared_error(y_test, dt_y_pred), mean_squared_error(y_test, rf_y_pred), mean_squared_error(y_test, xgbr2_y_pred), mean_squared_error(y_test, lgbm_y_predict)], 'R2 score': [r2_score(y_test, lr_y_pred), r2_score(y_test, dt_y_pred), r2_score(y_test, rf_y_pred), r2_score(y_test, xgbr2_y_pred), r2_score(y_test, lgbm_y_predict)]})
result_df

,Model,RMSE,MSE,R2 score
0,Linear regression,4.591985,21.086330,0.764062
1,Decision Tree,5.447574,29.676060,0.667950
2,Random forest,3.739248,13.981978,0.843554
3,XGBoost,3.648312,13.310180,0.851071
4,Light GBM,3.760545,14.141702,0.841766


In [28]:
# pickle.dump(grid_search, open("xgb_model.pkl", "wb"))